## Pacotes

In [1]:
import numpy as np

## Kalman Fusion

In [2]:
import numpy as np

class KalmanFusion3C:
    def __init__(self, Q=0.01, R=0.1, weights=(0.0, 0.5, 1.0)):
        """
        Filtro de Kalman 1D para fusão de probabilidade de 3 classes + heurísticas.

        Args:
            Q: variância do processo (incerteza do modelo GNN)
            R: variância da medida (incerteza dos estimadores heurísticos)
            weights: tupla de pesos para converter (baixa, media, alta) em score contínuo
                     padrão = (0, 0.5, 1)
        """
        self.Q = Q
        self.R = R
        self.weights = weights
        self.x = None  # estado estimado (score fundido)
        self.P = 1.0   # covariância inicial

    def _probs_to_score(self, probs):
        """
        Converte dict {'baixa':p1,'media':p2,'alta':p3} em score contínuo [0,1].
        """
        w_baixa, w_media, w_alta = self.weights
        return (
            w_baixa * probs.get("baixa", 0.0) +
            w_media * probs.get("media", 0.0) +
            w_alta  * probs.get("alta", 0.0)
        )

    def update(self, probs_dict, heuristics_cost):
        """
        Atualiza o estado do filtro.

        Args:
            probs_dict: dict com {'baixa':p1,'media':p2,'alta':p3}
            heuristics_cost: custo heurístico normalizado em [0,1]
        
        Returns:
            valor fundido (score entre 0 e 1)
        """
        # Converte as probabilidades em score
        model_score = self._probs_to_score(probs_dict)

        # Predição
        if self.x is None:
            self.x = model_score
        x_pred = self.x
        P_pred = self.P + self.Q

        # Medida
        z = heuristics_cost

        # Ganho de Kalman
        K = P_pred / (P_pred + self.R)

        # Atualização do estado
        self.x = x_pred + K * (z - x_pred)

        # Atualização da incerteza
        self.P = (1 - K) * P_pred

        return self.x

    def fused_cost(self):
        """
        Retorna o custo final (transforma score em custo aditivo).
        """
        return -np.log(self.x + 1e-6)

## Execução

In [3]:
# Saída do modelo
probs = {'baixa': 0.163, 'media': 0.307, 'alta': 0.529}

# Exemplo de custo heurístico normalizado (RSSI/ETX/Delay/Perda)
heuristics_cost = 0.35

# Instancia filtro
kf = KalmanFusion3C(Q=0.01, R=0.05)

# Atualiza com dados
fused_value = kf.update(probs, heuristics_cost)
fused_cost = kf.fused_cost()

In [4]:
print("Score fundido (Kalman):", fused_value)
print("Custo final:", fused_cost)

Score fundido (Kalman): 0.3656839622641509
Custo final: 1.0059830750874976
